In [2]:
import numpy as np
import pandas as pd
import math

In [3]:
def pairwise_dist(x, y):  # [5 pts]
    """
        Args:
            x: N x D numpy array
            y: M x D numpy array
        Return:
                dist: N x M array, where dist2[i, j] is the euclidean distance between
                x[i, :] and y[j, :]
    """
    #raise NotImplementedError
    return np.linalg.norm(x[:, None, :] - y[None, :, :], axis=-1)

In [5]:
cities = pd.read_csv("data/ds2.csv")
cities = cities.rename(columns = {'Unnamed: 0':'city_id'})
cities.head()

,city_id,State Abbreviation,Average Household Income 2020/2021,"Percent Change in Household Income, 1984-2019","Average Yearly Percent Change in Household Income, 1984-2019",MinWage,City,Average House Price 2020/2021,"Percent Change in Housing Prices, 2000-2019","Average Yearly Percent Change in Housing Prices, 2000-2019",...,Walk,OtherTransp,WorkAtHome,MeanCommute,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,Population
0,0,NY,-0.136317,0.851226,0.851226,0.855108,New York,1.045961,1.230131,1.230131,...,1.627811,-0.553042,-1.312925,3.828220,-0.203596,0.226288,0.006249,-0.591108,-0.016201,23.673822
1,1,CA,0.737962,0.014144,0.014144,1.139948,Los Angeles,1.393001,1.882940,1.882940,...,0.092612,0.256218,0.250712,1.128201,-0.239062,-0.512411,1.786000,0.071967,0.417355,10.737820
2,2,IL,0.457552,0.224546,0.224546,0.427849,Chicago,-0.363198,-0.868798,-0.868798,...,0.845506,0.347146,-0.302575,1.687189,0.671963,-0.320016,-0.911262,-0.591108,1.564181,7.403277
3,3,TX,-0.416098,-0.755247,-0.755247,-1.263387,Houston,-0.588687,-0.388227,-0.388227,...,-0.560289,-0.034752,-0.771666,0.529134,0.669746,-0.873447,0.359989,-0.093802,-0.132749,5.907363
4,4,PA,-0.042637,1.850164,1.850164,-1.263387,Philadelphia,-0.602243,0.856306,0.856306,...,3.374761,1.256428,-0.807750,1.533204,0.654230,-0.168001,-1.204202,-1.088414,2.240155,4.068471


In [10]:
data_us = pd.read_csv('data/uscities.csv')

data_us = data_us[['city', 'state_id', 'state_name', 'county_name', 'lat', 'lng']]
data_us['city_name_id'] = data_us.apply(lambda x: x['city'] + ", " + x['state_name'] + ", " + x['county_name'] + " County", axis = 1)
data_us['city_state_id'] = data_us.apply(lambda x: x['city'] + ", " + x['state_name'], axis = 1)

data_cities = cities.copy()
data_cities['city_name_id'] = data_cities.apply(lambda x: x['City'] + ", " + x['State'] + ", " + x['County'], axis = 1)
data_cities['city_state_id'] = data_cities.apply(lambda x: x['City'] + ", " + x['State'], axis = 1)

city_all = data_cities.merge(data_us, on = 'city_name_id', how = 'left')
city_all = city_all.rename(columns = {'city_state_id_x':'city_state'})
city_all.dropna(inplace = True)
city_all = city_all.drop(columns = ['city_id', 'city_state_id_y'])
city_all = city_all.reset_index(drop = True)
city_all = city_all.reset_index()
city_all = city_all.rename(columns = {'index':'city_id'})
city_all.to_csv('data/city_all.csv', index = False)
city_all.head()

,city_id,State Abbreviation,Average Household Income 2020/2021,"Percent Change in Household Income, 1984-2019","Average Yearly Percent Change in Household Income, 1984-2019",MinWage,City,Average House Price 2020/2021,"Percent Change in Housing Prices, 2000-2019","Average Yearly Percent Change in Housing Prices, 2000-2019",...,Unemployment,Population,city_name_id,city_state,city,state_id,state_name,county_name,lat,lng
0,0,NY,-0.136317,0.851226,0.851226,0.855108,New York,1.045961,1.230131,1.230131,...,-0.016201,23.673822,"New York, New York, Queens County","New York, New York",New York,NY,New York,Queens,40.6943,-73.9249
1,1,CA,0.737962,0.014144,0.014144,1.139948,Los Angeles,1.393001,1.882940,1.882940,...,0.417355,10.737820,"Los Angeles, California, Los Angeles County","Los Angeles, California",Los Angeles,CA,California,Los Angeles,34.1141,-118.4068
2,2,TX,-0.416098,-0.755247,-0.755247,-1.263387,Houston,-0.588687,-0.388227,-0.388227,...,-0.132749,5.907363,"Houston, Texas, Harris County","Houston, Texas",Houston,TX,Texas,Harris,29.7860,-95.3885
3,3,AZ,-0.317483,1.027372,1.027372,0.712688,Phoenix,-0.258789,-0.013201,-0.013201,...,-0.314562,3.954610,"Phoenix, Arizona, Maricopa County","Phoenix, Arizona",Phoenix,AZ,Arizona,Maricopa,33.5722,-112.0892
4,4,TX,-0.416098,-0.755247,-0.755247,-1.263387,San Antonio,-0.534323,-0.280991,-0.280991,...,-0.263282,3.656073,"San Antonio, Texas, Bexar County","San Antonio, Texas",San Antonio,TX,Texas,Bexar,29.4632,-98.5238


In [11]:
city_all.columns

Index(['city_id', 'State Abbreviation', 'Average Household Income 2020/2021',
       'Percent Change in Household Income, 1984-2019',
       'Average Yearly Percent Change in Household Income, 1984-2019',
       'MinWage', 'City', 'Average House Price 2020/2021',
       'Percent Change in Housing Prices, 2000-2019',
       'Average Yearly Percent Change in Housing Prices, 2000-2019',
       'Violent Crime Total', 'Property Crime Total',
       'Property Crime Per Capita', 'Violent Crime Per Capita',
       'Total Crime Per Capita', 'State', 'County', 'TotalPop', 'Men', 'Women',
       'VotingAgeCitizen', 'Employed', 'Hispanic', 'White', 'Black', 'Native',
       'Asian', 'Pacific', 'Income', 'IncomePerCap', 'Poverty', 'ChildPoverty',
       'Professional', 'Service', 'Office', 'Construction', 'Production',
       'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome',
       'MeanCommute', 'PrivateWork', 'PublicWork', 'SelfEmployed',
       'FamilyWork', 'Unemployment', 'Po

In [26]:
meta_columns = ['city_id', 'State', 'County', 'State Abbreviation', 'city_name_id', 'city_state', 'city', 'City', 'state_id', 'county_name', 'state_name', 'lat', 'lng']
city_metadata = city_all[meta_columns]
city_metadata.to_csv('data/city_metadata.csv', index = False)


In [14]:
def id_to_city(city_id):
    return pd.read_csv('data/city_metadata.csv').iloc[[city_id]]
id_to_city(10)

,city_id,State Abbreviation,city_name_id,city_state,city,state_id,county_name,lat,lng
10,10,CA,"San Francisco, California, San Francisco County","San Francisco, California",San Francisco,CA,San Francisco,37.7558,-122.4449


In [27]:
city_df = city_all.drop(columns = meta_columns)
# delete_cols = ['city_state_id_y', 'state_id', 'city_name_id', 'city', 'state_name', 'county_name']
# city_df = city_df.drop(columns = delete_cols)
city_df.head()

,Average Household Income 2020/2021,"Percent Change in Household Income, 1984-2019","Average Yearly Percent Change in Household Income, 1984-2019",MinWage,Average House Price 2020/2021,"Percent Change in Housing Prices, 2000-2019","Average Yearly Percent Change in Housing Prices, 2000-2019",Violent Crime Total,Property Crime Total,Property Crime Per Capita,...,Walk,OtherTransp,WorkAtHome,MeanCommute,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,Population
0,-0.136317,0.851226,0.851226,0.855108,1.045961,1.230131,1.230131,18.378204,14.963376,-0.662457,...,1.627811,-0.553042,-1.312925,3.828220,-0.203596,0.226288,0.006249,-0.591108,-0.016201,23.673822
1,0.737962,0.014144,0.014144,1.139948,1.393001,1.882940,1.882940,10.946839,9.795925,-0.165898,...,0.092612,0.256218,0.250712,1.128201,-0.239062,-0.512411,1.786000,0.071967,0.417355,10.737820
2,-0.416098,-0.755247,-0.755247,-1.263387,-0.588687,-0.388227,-0.388227,10.953656,11.118278,1.514108,...,-0.560289,-0.034752,-0.771666,0.529134,0.669746,-0.873447,0.359989,-0.093802,-0.132749,5.907363
3,-0.317483,1.027372,1.027372,0.712688,-0.258789,-0.013201,-0.013201,5.011719,5.650089,0.542938,...,-0.595581,0.556281,1.086656,0.012334,0.507934,-0.571792,0.072575,-0.259570,-0.314562,3.954610
4,-0.416098,-0.755247,-0.755247,-1.263387,-0.534323,-0.280991,-0.280991,3.934625,6.799020,1.237460,...,-0.489705,-0.589414,-0.609289,-0.188058,0.088996,0.031519,-0.270109,-0.756876,-0.263282,3.656073


In [28]:
city_columns = list(enumerate(city_df.columns))
city_columns

[(0, 'Average Household Income 2020/2021'),
 (1, 'Percent Change in Household Income, 1984-2019'),
 (2, 'Average Yearly Percent Change in Household Income, 1984-2019'),
 (3, 'MinWage'),
 (4, 'Average House Price 2020/2021'),
 (5, 'Percent Change in Housing Prices, 2000-2019'),
 (6, 'Average Yearly Percent Change in Housing Prices, 2000-2019'),
 (7, 'Violent Crime Total'),
 (8, 'Property Crime Total'),
 (9, 'Property Crime Per Capita'),
 (10, 'Violent Crime Per Capita'),
 (11, 'Total Crime Per Capita'),
 (12, 'TotalPop'),
 (13, 'Men'),
 (14, 'Women'),
 (15, 'VotingAgeCitizen'),
 (16, 'Employed'),
 (17, 'Hispanic'),
 (18, 'White'),
 (19, 'Black'),
 (20, 'Native'),
 (21, 'Asian'),
 (22, 'Pacific'),
 (23, 'Income'),
 (24, 'IncomePerCap'),
 (25, 'Poverty'),
 (26, 'ChildPoverty'),
 (27, 'Professional'),
 (28, 'Service'),
 (29, 'Office'),
 (30, 'Construction'),
 (31, 'Production'),
 (32, 'Drive'),
 (33, 'Carpool'),
 (34, 'Transit'),
 (35, 'Walk'),
 (36, 'OtherTransp'),
 (37, 'WorkAtHome'),
 (

In [29]:
city_np = np.array(city_df)
city_np.shape

(692, 45)

In [30]:
city_np

array([[-1.36317125e-01,  8.51225950e-01,  8.51225950e-01, ...,
        -5.91107781e-01, -1.62011835e-02,  2.36738216e+01],
       [ 7.37961649e-01,  1.41441493e-02,  1.41441493e-02, ...,
         7.19670230e-02,  4.17354867e-01,  1.07378196e+01],
       [-4.16097945e-01, -7.55247430e-01, -7.55247430e-01, ...,
        -9.38016781e-02, -1.32748509e-01,  5.90736280e+00],
       ...,
       [ 7.11106174e-01,  1.89021433e+00,  1.89021433e+00, ...,
        -2.59570379e-01, -1.88691225e-01, -2.69061550e-01],
       [ 1.35302460e+00,  2.79941865e+00,  2.79941865e+00, ...,
         2.37735724e-01, -1.24694094e+00, -2.69370536e-01],
       [-1.32623241e+00, -6.54708389e-01, -6.54708389e-01, ...,
         1.23234793e+00, -2.44633941e-01, -2.69596553e-01]])

In [31]:
cities_dist = None
cities_nn = None
if True:
    cities_dist = pairwise_dist(city_np, city_np) #.tofile('data/pairwise_city_dist.csv', sep = ',')
    #np.savetxt("data/city_pairwise_dist.csv", cities_dist, delimiter=",")
    temp = pd.DataFrame(cities_dist)
    temp.to_csv("data/city_distances.csv", index = False)
    cities_nn = pd.DataFrame(np.argsort(cities_dist))
    #np.savetxt("data/city_nn_ranked.csv", cities_nn, delimiter=",")
    cities_nn.to_csv("data/city_nn.csv", index = False)
    cities_dist = temp
    del temp
else:
    cities_dist = np.array(pd.read_csv("data/city_distances.csv"))
    cities_nn = np.array(pd.read_csv("data/city_nn.csv"))

In [32]:
np.sum(cities_nn, axis = 1)

0      239086
1      239086
2      239086
3      239086
4      239086
        ...  
687    239086
688    239086
689    239086
690    239086
691    239086
Length: 692, dtype: int64

In [34]:
def get_nn(city_id, n = 5):
    """
        Args:
            city_id: int
            n: int
        Return:
            nearest_neighbors: (n,) ndarray, where nearest_neighbors[i] is the city_id of the i'th most similar city
    """

    #Don't include the first column since it will always be itself (distances between a city and itself is 0 miles)
    dta = cities_nn.iloc[city_id, 1:n+1]

    return dta
    
get_nn(25, 5)

1    118
2    227
3    238
4    311
5    173
Name: 25, dtype: int64

In [35]:
cities_nn.head()

,0,1,2,3,4,5,6,7,8,9,...,682,683,684,685,686,687,688,689,690,691
0,0,1,2,3,4,6,5,10,16,17,...,491,470,353,642,560,515,316,569,660,40
1,1,2,3,4,6,5,25,16,12,8,...,353,642,515,86,560,470,316,569,660,40
2,2,4,6,1,3,15,12,17,13,9,...,642,421,436,676,480,483,470,367,0,40
3,3,6,4,5,12,9,27,8,22,11,...,86,316,660,10,676,483,569,367,40,0
4,4,6,3,12,9,19,11,8,26,17,...,421,676,483,470,436,480,10,367,40,0


In [37]:
city_metadata.columns

Index(['city_id', 'State', 'County', 'State Abbreviation', 'city_name_id',
       'city_state', 'city', 'City', 'state_id', 'county_name', 'state_name',
       'lat', 'lng'],
      dtype='object')

In [38]:
nearest_neighbors = pd.concat([city_metadata, cities_nn.iloc[:, 1:21]], axis = 1)

nearest_neighbors['city_county'] = nearest_neighbors.apply(lambda x: x['City'] + ', ' + x['County'], axis = 1)
nearest_neighbors['city_st_abbr'] = nearest_neighbors.apply(lambda x: x['City'] + ', ' + x['State Abbreviation'], axis = 1)

nearest_neighbors.to_csv('data/nearest_neighbors.csv', index = False)

In [39]:
pd.read_csv('data/nearest_neighbors.csv')

,city_id,State,County,State Abbreviation,city_name_id,city_state,city,City,state_id,county_name,...,13,14,15,16,17,18,19,20,city_county,city_st_abbr
0,0,New York,Queens County,NY,"New York, New York, Queens County","New York, New York",New York,New York,NY,Queens,...,18,13,21,7,11,20,33,12,"New York, Queens County","New York, NY"
1,1,California,Los Angeles County,CA,"Los Angeles, California, Los Angeles County","Los Angeles, California",Los Angeles,Los Angeles,CA,Los Angeles,...,9,218,173,134,85,112,7,227,"Los Angeles, Los Angeles County","Los Angeles, CA"
2,2,Texas,Harris County,TX,"Houston, Texas, Harris County","Houston, Texas",Houston,Houston,TX,Harris,...,26,19,5,35,16,34,22,36,"Houston, Harris County","Houston, TX"
3,3,Arizona,Maricopa County,AZ,"Phoenix, Arizona, Maricopa County","Phoenix, Arizona",Phoenix,Phoenix,AZ,Maricopa,...,26,34,19,69,57,30,21,303,"Phoenix, Maricopa County","Phoenix, AZ"
4,4,Texas,Bexar County,TX,"San Antonio, Texas, Bexar County","San Antonio, Texas",San Antonio,San Antonio,TX,Bexar,...,35,21,60,5,56,29,23,48,"San Antonio, Bexar County","San Antonio, TX"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,687,Wisconsin,Milwaukee County,WI,"Greenfield, Wisconsin, Milwaukee County","Greenfield, Wisconsin",Greenfield,Greenfield,WI,Milwaukee,...,211,300,488,573,552,682,296,669,"Greenfield, Milwaukee County","Greenfield, WI"
688,688,California,Los Angeles County,CA,"Monrovia, California, Los Angeles County","Monrovia, California",Monrovia,Monrovia,CA,Los Angeles,...,170,513,270,478,124,475,193,168,"Monrovia, Los Angeles County","Monrovia, CA"
689,689,Oregon,Marion County,OR,"Keizer, Oregon, Marion County","Keizer, Oregon",Keizer,Keizer,OR,Marion,...,230,391,243,387,167,690,320,528,"Keizer, Marion County","Keizer, OR"
690,690,Utah,Utah County,UT,"Spanish Fork, Utah, Utah County","Spanish Fork, Utah",Spanish Fork,Spanish Fork,UT,Utah,...,233,520,177,689,678,273,610,509,"Spanish Fork, Utah County","Spanish Fork, UT"
